In [1]:
import numpy as np
import matplotlib.pyplot as plt
import posfai2 as my
import time
import scipy.sparse as sp

In [2]:
# Define model
L=1000; alpha=4; n_order = 4; n_adj = 4; alphabet='ACGT';
ohe_spec_str = my.get_ohe_spec_str(L, n_order=n_order, n_adj=n_adj)

In [3]:
seq = my.make_random_seqs(L=L, N=1, alphabet=alphabet)[0]
seq

'TCTGCGTTACCATGCAATATATCGCAAGACGGCCACGTTTAAATGAACAGATTGGGGGCAAAGCAGGACTCTCGCGGTGTACCACAATAGTCCTGTGTGCGCATTAGTTAGGCAGAACAGTGGCTAGTTTGGGCCCTTGCTCCATATGTGGACAAGACAAGCAAATCACTTCGCAGTCCGTTGATAGTGAACACAGCCACTTGGTTCCAGATACGGTGCGCATTATGAAACCAGTATAGGGCGAGACTAATCTCACCAAATGCCCCTACCGTGGTCGTTCTACTTGCCGATGGGAAACATGCTTTTATCTCTTGAAGCCAACGTGGAACCTCGGAGCGGTTGAGTTACCACGTTCTCCAAATGATACTACCATTTACCCTGATCTGCGGCGCCGCTATCGTTGTTTTGAGGTTTTGCTAGTTGAGGAGGTTGCTAAGGTAACTCCATAAGGGTGTCCGTCTATATGGGGAAACCAGGAATTAGCAGGTATATTCGCAAGGTAATTCACCGACGGGTGTTCGAGGTTAGTCTTGTCGACCGGCCGCGACGGATGGGATCTTGACATCAACTGCAAACAGCAAACCTGCCGCTCTTACTGCAATTCTGCATTCTCCTCCGTTTTTTCGTATGTCAAATACTCTTAATAAGTTTACTCGAAAAATATAGAGGAGGCTATAGGCTACGGAGTGCCCTTTTTGATGGGACTCCACACGTAGCCGCAGTGAACGGGCGCCATGTGGATGGCCCATACTAGCGGGATCGTAATGCAATGTTCCAGGTGCGCTGAAGGGTCGCATTTACCCACCGGGTCGGACCCGGAGTTCGCTGTATGTCGTGTGTTAAGGTGTCACCTAGCGCTCCTATCACCGAGAGGAGGTAGGACGAGAAGATTTGCCATAGGACAATCGTTCTCTACTGCGCGATCTGCAATTGTCTGAGCTATCGAGGTACTCATGGATCAGTAGTTTGAATGGCGGCGGTAGCAGCACAGTTGCAGAC

In [4]:
x_ohe = my.seq_to_x_ohe(seq=seq, ohe_spec_str=ohe_spec_str, sparse=True)
x_ohe

<498625x1 sparse matrix of type '<class 'numpy.float64'>'
	with 7984 stored elements in Compressed Sparse Column format>

In [5]:
timing_dict = {}

### Start timing
start_time = time.perf_counter()

# Get transformation matrix
print('ohe_spec_to_T...')
t0 = time.perf_counter()
T = my.ohe_spec_to_T(ohe_spec_str, alpha=alpha, compute_inv=False)
timing_dict['ohe_spec_to_T'] =  time.perf_counter() - t0

# Get reordering matrix
print('ohe_spec_to_B...')
t0 = time.perf_counter()
B, B_inv = my.ohe_spec_to_B(ohe_spec_str, alpha=alpha)
timing_dict['ohe_spec_to_B'] = time.perf_counter() - t0

# Get sim_spec
print('ohe_to_sim_spec...')
t0 = time.perf_counter()
sim_spec_str = my.ohe_to_sim_spec(ohe_spec_str)
timing_dict['ohe_to_sim_spec'] = time.perf_counter() - t0

# Get thinning matrix
print('get_thinning_matrix...')
t0 = time.perf_counter()
A, A_inv = my.get_thinning_matrix(sim_spec_str, alpha=alpha)
timing_dict['get_thinning_matrix'] = time.perf_counter() - t0

# Get distilling matrix
print('get_distilling_matrix...')
t0 = time.perf_counter()
D, D_inv, gamma = my.get_distilling_matrix(sim_spec_str, alpha=alpha)
timing_dict['get_distilling_matrix'] = time.perf_counter() - t0

# Get gauge basis
print('E computation...')
t0 = time.perf_counter()
E = D @ A @ B @ T
gauge_basis = E[-gamma:, :].T
timing_dict['E computation'] = time.perf_counter() - t0

### End timming
end_time = time.perf_counter()

# Display results:
elapsed_time = end_time - start_time
print(f'---------------------------\nResults')
print(f'L: {L}')
print(f'alpha: {alpha}')
print(f'n_order: {n_order}')
print(f'n_adj: {n_adj}')
print(f'ohe_spec_str: {ohe_spec_str[:80]}...')
print(f'sim_spec_str: {sim_spec_str[:80]}...')

M = T.shape[0]
print(f'M: {M:,d}')
print(f'gamma: {gamma:,d}')
print('Unique elements of gauge basis', np.unique(gauge_basis.data))

print(f"The code took {elapsed_time:.3f} sec.")
for key, val in timing_dict.items():
    print(f"\t{key}: {val:.4f} sec.")

# Objects of interest:
objs_dict = {'    T':T,
             '    A':A,
             '    B':B,
             '    D':D,
             '    E':E,
             'gauge_basis':gauge_basis}
for key, val in objs_dict.items():
    size = val.data.nbytes
    pct = 100*size/(M*M)
    print(f"\t{key}: {size:10,d} bytes, {pct:.3f}% dense.")

ohe_spec_to_T...
ohe_spec_to_B...
ohe_to_sim_spec...
get_thinning_matrix...
get_distilling_matrix...
E computation...
---------------------------
Results
L: 1000
alpha: 4
n_order: 4
n_adj: 4
ohe_spec_str: .+0+1+2+3+4+5+6+7+8+9+10+11+12+13+14+15+16+17+18+19+20+21+22+23+24+25+26+27+28+2...
sim_spec_str: .+.+0+.+1+.+2+.+3+.+4+.+5+.+6+.+7+.+8+.+9+.+10+.+11+.+12+.+13+.+14+.+15+.+16+.+1...
M: 498,625
gamma: 307,137
Unique elements of gauge basis [-1  1]
The code took 3.210 sec.
	ohe_spec_to_T: 1.8773 sec.
	ohe_spec_to_B: 0.3512 sec.
	ohe_to_sim_spec: 0.0590 sec.
	get_thinning_matrix: 0.3028 sec.
	get_distilling_matrix: 0.1301 sec.
	E computation: 0.4888 sec.
	    T: 627,016,712 bytes, 0.252% dense.
	    A:  6,446,096 bytes, 0.003% dense.
	    B:  3,989,000 bytes, 0.002% dense.
	    D:  3,989,000 bytes, 0.002% dense.
	    E: 118,343,936 bytes, 0.048% dense.
	gauge_basis: 101,926,584 bytes, 0.041% dense.


In [6]:
# Make sure gauge basis is actually orthogonal to xs. 

# Randomly sample gauge vectors
num_gauge_vecs = gamma
gauge_vec_ix = np.random.choice(a=gamma, size=num_gauge_vecs, replace=False)
gauge_vecs = gauge_basis[:,gauge_vec_ix]

# Randomly sample sequences and encode them
num_rand_seqs = 10
rand_seqs = my.make_random_seqs(L, N=num_rand_seqs, alphabet=alphabet)
x_ohes = sp.hstack([my.seq_to_x_ohe(seq, ohe_spec_str, alphabet=alphabet) for seq in rand_seqs])
dots = (gauge_vecs.T@x_ohes).todense()

# Take dot products and make sure all are zero
print(f'{len(dots.ravel()):,} dot products between {num_gauge_vecs:,} gauge vectors and encodings of {num_rand_seqs:,} random seqs are all zero: {np.allclose(dots, 0)}')
print('Unique nonzero elements of gauge basis', np.unique(gauge_basis.data))

3,071,370 dot products between 307,137 gauge vectors and encodings of 10 random seqs are all zero: True
Unique nonzero elements of gauge basis [-1  1]
